In [1]:
import os
import requests
from typing import Dict, Optional

In [2]:
class LLMClient:
    
    def __init__(
        self,
        api_key: str = "cse476",
        api_base: str = "http://10.4.58.53:41701/v1",
        model: str = "bens_model"
    ):

        self.api_key = api_key
        self.api_base = api_base
        self.model = model
        self.call_count = 0
    
    def call(
        self,
        prompt: str,
        system: str = "You are a helpful assistant.",
        temperature: float = 0.0,
        max_tokens: int = 128,
        timeout: int = 60
    ) -> Dict:
        self.call_count += 1
        
        url = f"{self.api_base}/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }
        payload = {
            "model": self.model,
            "messages": [
                {"role": "system", "content": system},
                {"role": "user", "content": prompt}
            ],
            "temperature": temperature,
            "max_tokens": max_tokens,
        }
        
        try:
            resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
            status = resp.status_code
            
            if status == 200:
                data = resp.json()
                text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
                return {
                    "ok": True,
                    "text": text,
                    "status": status,
                    "error": None,
                    "raw": data
                }
            else:
                try:
                    err_text = resp.json()
                except Exception:
                    err_text = resp.text
                
                return {
                    "ok": False,
                    "text": None,
                    "status": status,
                    "error": str(err_text),
                    "raw": None
                }
                
        except requests.RequestException as e:
            return {
                "ok": False,
                "text": None,
                "status": -1,
                "error": str(e),
                "raw": None
            }
    
    def reset_counter(self):
        self.call_count = 0
    
    def get_call_count(self) -> int:
        return self.call_count


def call_model_chat_completions(
    prompt: str,
    system: str = "You are a helpful assistant.",
    model: str = "bens_model",
    temperature: float = 0.0,
    timeout: int = 60
) -> Dict:
    client = LLMClient(model=model)
    return client.call(prompt, system, temperature, timeout=timeout)

In [3]:
if __name__ == "__main__":
    print("Testing LLM Client:")
    
    client = LLMClient()
    
    print("\nTest 1: Simple Math")
    result = client.call("What is 5 + 3? Answer with just the number.")
    
    if result["ok"]:
        print(f"Success! Answer: {result['text']}")
        print(f"   API calls made: {client.get_call_count()}")
    else:
        print(f"Failed: {result['error']}")

    print("\nTest 2: Common Sense")
    result = client.call(
        prompt="You place an ice cube in a glass of water and mark the water level. After the ice melts, does the water level rise, fall, or stay the same? ",
        system="Answer with exactly one of: 'rise', 'fall', 'stay the same'."
    )
    
    if result["ok"]:
        print(f"Success! Answer: {result['text']}")
        print(f"   API calls made: {client.get_call_count()}")
    else:
        print(f"Failed: {result['error']}")
        
    print(f"Total API calls: {client.get_call_count()}")

    print("\nTest 3: logic_race")
    result = client.call(
        prompt="In a race, you pass the person in second place. What position are you now in? ",
        system="Answer with a single word like 'first', 'second', 'third'."
    )
    
    if result["ok"]:
        print(f"Success! Answer: {result['text']}")
        print(f"   API calls made: {client.get_call_count()}")
    else:
        print(f"Failed: {result['error']}")
        
    print(f"Total API calls: {client.get_call_count()}")

Testing LLM Client:

Test 1: Simple Math
Success! Answer: 8
   API calls made: 1

Test 2: Common Sense
Success! Answer: stay the same
   API calls made: 2
Total API calls: 2

Test 3: logic_race
Success! Answer: second
   API calls made: 3
Total API calls: 3
